In [26]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [27]:
import pandas as pd
df = pd.read_csv(r'/Users/arinatryaskova/Documents/quora_duplicate_questions.tsv', sep='\t')

In [29]:
df = df.drop(df[df.is_duplicate == 0].index)

In [50]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [47]:
corpus = df['question1'].to_numpy()[:10000]
questions2 = df['question2'].to_numpy()[:10000]

In [48]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [49]:
# Query sentences:
queries = questions2

score_counter = 0

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 1
for query, sentence in zip(queries, corpus):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()
    
    top_results = torch.topk(cos_scores, k=top_k)

    for score, idx in zip(top_results[0], top_results[1]):
        if corpus[idx] == sentence:
            score_counter += 1
        
print(good_counter / len(corpus))

0.5983
